In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

In [2]:
model = AutoModelForCausalLM.from_pretrained('EleutherAI/polyglot-ko-1.3b')
tokenizer = AutoTokenizer.from_pretrained(
    'EleutherAI/polyglot-ko-1.3b', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

data_path_1_SFT = './data_kochatgpt/kochatgpt_1_SFT.jsonl'
print(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PreTrainedTokenizerFast(name_or_path='EleutherAI/polyglot-ko-1.3b', vocab_size=30000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>', 'additional_special_tokens': ['<|endoftext|>', '<|sep|>', '<|acc|>', '<|tel|>', '<|rrn|>']}, clean_up_tokenization_spaces=True)


In [3]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [4]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [5]:
train_dataset = SFT_dataset(data_path_1_SFT=data_path_1_SFT, tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s' % train_dataset.input_ids[0])
print('output: %s' % train_dataset.labels[0])

input : tensor([    6,     6,     6, 10276,  3864, 23184, 29828,    11,  7262,   348,
           12,    29,   202,  1198,  3648,   463,  5992,  8649,  2733,    34,
          202,   202,     6,     6,     6,  3069,  5862,    83,  2600,   426,
           11,  9643,    12,    29,    10,   828,   272,  5626, 29377,  3620,
          270,   453,    15,  1455, 10562,  3754,   274,  2708,  1610,   301,
         1384,   283,   327,   295,   536,   827,    17,  1146,  1754, 10562,
        23342,   463,  5992,   272,  8649,    15, 11765,    15, 12305,   433,
         1186,   288,  5335,   285,  5992,   301,  1203,  2136,    17,  1146,
         8649,   272,   924,   407,   333,  4213, 19665,   286,  5037,   327,
          316,   818,   274,    15,  8649,   301,  1203,   284,   272,   955,
          309,   750,   827,    17,   768,   488, 15337,  4548,  1793,  1641,
          274,  1026,  8129,   365,   327,  2556,  2403,   288,  1610,  5829,
          726,   274,  1912,   310,   458,  9061, 10710,

In [6]:
model.config.use_cache = False

training_args = TrainingArguments(
    output_dir="./test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
    warmup_steps=5,
    prediction_loss_only=True,
    gradient_checkpointing=True,
    fp16=True,
    optim="adafactor",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [7]:
trainer.train()
model.save_pretrained('./output_2_SFT')

Step,Training Loss
500,1.708600
1000,1.591800
1500,1.552200


In [7]:
generator = pipeline('text-generation', model='./output_2_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:375 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:375 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:375 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:375 for open-end generation.



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 언어모델이며, 불고기용 고기 한우가 어떤 것인지 알 수 없습니다. 이에 대한 정보를 제공해주시면 더 정확한 답변을 드릴 수 있을 것 같습니다.\n감사합니다.!\n죄송합니다. 제가 이해하지 못했

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉스턴은 43대 부통령으로 재직한 년도에 대한 정보가 제공되지 않았기 때문에 정확한 답변을 제공할 수 없습니다. 추가적인 정보를 제공해 주시면 더 나은 답변을 드릴 수 있을 것입니다.\n\n감사합니다

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오헤 국제공항은 미국 일리노이주 시카고에 위치해 있습니다. 국제공항이라고도 불립니다. 국제공항에서 시카고 시내까지는 약 1시간 정도 소요됩니다. 국제공항의 위치는 다음과 같습니다.\n\n1. 시카고 오헤어 국제공항 (Chicago

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이므로 미세먼지에 대한 정보를 가지고 있지 않습니다. 하지만 보통 미세먼지 농도가 높으면 건강에 해로울 수 있으니 조심하시는 것이 좋을 것 같습니다. 또한, 마스크 착용 등 예방수칙을 준수하여 건강을 유지하시기 바랍니다.


In [8]:
torch.cuda.empty_cache()

In [ ]:
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    70W /  70W |  14846MiB / 15109MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
+-----------------------------------------------------------------------------+